In [2]:
import os
import zipfile
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np

# --- 1. Helper Functions and Quantizer Class ---
# A module to perform absmax quantization and dequantization
class AbsMaxQuantizer(nn.Module):
    def __init__(self, layer):
        super().__init__()
        self.layer = layer
        
        # Get weights and quantize them on initialization
        weights = layer.weight.data
        max_val = torch.max(torch.abs(weights))
        self.scale = 127.0 / max_val
        self.quantized_weights = torch.round(weights * self.scale).to(torch.int8)

    def forward(self, x):
        # De-quantize the weights for computation
        dequantized_weights = self.quantized_weights.to(torch.float32) / self.scale
        
        # Perform the original layer's forward pass with the de-quantized weights
        if isinstance(self.layer, nn.Conv2d):
            return nn.functional.conv2d(x, dequantized_weights, self.layer.bias, self.layer.stride, self.layer.padding)
        elif isinstance(self.layer, nn.Linear):
            return nn.functional.linear(x, dequantized_weights, self.layer.bias)
        else:
            return self.layer(x)

# --- 2. Model Loading and Quantization ---
print("Loading and quantizing VGG16 model...")
model_original = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
model_quantized = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model_original.to(device)
model_quantized.to(device)

for name, module in model_quantized.named_modules():
    if isinstance(module, (nn.Conv2d, nn.Linear)):
        path = name.split('.')
        parent = model_quantized
        for i in range(len(path) - 1):
            parent = parent._modules[path[i]]
        layer_name = path[-1]
        quantized_layer = AbsMaxQuantizer(module)
        setattr(parent, layer_name, quantized_layer)
print("Quantization complete.")

# --- 3. Data Extraction and Loading ---
print("Preparing test data...")
zip_file_path = "testABC.zip"
extracted_folder_name = "test_data"

if not os.path.exists(extracted_folder_name):
    print(f"Extracting {zip_file_path}...")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_folder_name)
else:
    print(f"Directory '{extracted_folder_name}' already exists. Skipping extraction.")

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_dataset = datasets.ImageFolder(root=extracted_folder_name, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
print("Test data loaded successfully.")

# --- 4. Consistency and Accuracy Calculation ---
print("Starting consistency and accuracy check...")
model_original.eval()
model_quantized.eval()

consistency_matches = 0
original_correct = 0
quantized_correct = 0
total_images = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        output_original = model_original(images)
        _, preds_original = torch.max(output_original, 1)

        output_quantized = model_quantized(images)
        _, preds_quantized = torch.max(output_quantized, 1)

        consistency_matches += (preds_original == preds_quantized).sum().item()
        original_correct += (preds_original == labels).sum().item()
        quantized_correct += (preds_quantized == labels).sum().item()

        total_images += images.size(0)

consistency_percentage = (consistency_matches / total_images) * 100
original_accuracy = (original_correct / total_images) * 100
quantized_accuracy = (quantized_correct / total_images) * 100

print("\n--- Final Results ---")
print(f"Top-1 Prediction Consistency: {consistency_percentage:.2f}%")


Loading and quantizing VGG16 model...
Using device: cuda
Quantization complete.
Preparing test data...
Directory 'test_data' already exists. Skipping extraction.
Test data loaded successfully.
Starting consistency and accuracy check...

--- Final Results ---
Top-1 Prediction Consistency: 95.24%
